# From raw data to dataloader

In [ ]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part
vocab_loc = "/Path/to/pre_trained/scBERT/vocab_gene2vec_16906.pkl"
#----> data part
csv_loc = "/Path/to/allMCF10data_batches_labelled_with_pseudotimes_normalised.csv"
adata_loc = "Path/to/allMCF.h5ad"

import pickle
with open(vocab_loc, "rb") as f:
    vocab = pickle.load(f)

In [ ]:
# init preprocessor
from scLLM.Dataset.preprocessor import Preprocessor
from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT
dataset_para = Dataset_para(gene_vocab=vocab,
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,
                            )

preprocess = Preprocessor(dataset_para)

In [ ]:

# load data with csv file that you want to preprocess
import pandas as pd
df = pd.read_csv(csv_loc)

# gene_name_mask: gene name mask if this colume in csv file should be included, then
#                            we set this gene name with True
gene_name_mask = [True if i>=2 and i<2044 else False for i in range(len(list(df.keys())))]
# sample_id_idx: a index in a csv file that can use to sample items 
sample_id_idx = "Row.names"
# sample_mask: sample mask if this row in csv file should be included, then
sample_mask = [True for i in range(df.shape[0])] # use all cases
# obs_name_mask: label name in csv file.if this colume include a label, set ture
obs_name_mask = [True if i==len(list(df.keys()))-1 else False for i in range(len(list(df.keys())))]

preprocess.from_csv(
    csv_loc = csv_loc,
    gene_name_mask = gene_name_mask,
    sample_id_idx = sample_id_idx,
    sample_mask = sample_mask,
    obs_name_mask=obs_name_mask,
    df=df# optional
)
preprocess.run()
preprocess.save_adata(adata_loc)


In [ ]:
preprocess.load_adata(adata_loc)

In [ ]:
data = preprocess.to_data(data_type="log1p")
label,class_weight = preprocess.to_label(
                          label_key="pseudotimes",
                          binarize=True,
                          bin_nb=20,)

In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit, StratifiedShuffleSplit, StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2023)

In [ ]:
idx_tr,idx_val = next(iter(sss.split(data, label)))

In [ ]:
data_train, label_train = data[idx_tr], label[idx_tr]

In [ ]:
data_train.shape,label_train.shape

In [ ]:
from scLLM.Dataset.dataset import SCDataset
train_dataset = SCDataset(data_train, label_train)

In [ ]:
train_loader = preprocess.to_dataloader(train_dataset)